In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

## First Optimization Attempt
### Focus on dropping features for Model improvement

In [ ]:
# Still drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],1)
application_df.head()

In [ ]:
# Review Features
application_df.nunique()

In [ ]:
application_df['STATUS'].value_counts()

In [ ]:
application_df['SPECIAL_CONSIDERATIONS'].value_counts()

In [ ]:
# Features to remove from analysis
"""
Reasons for removing from analysis:
STATUS: 34294 to 5 in value_counts, even if informative, model will not learn association
SPECIAL_CONSIDERATIONS: 34272 to 27 in value_counts, even if informative, model will not learn association
"""
remove_features = ['STATUS','SPECIAL_CONSIDERATIONS']
application_df = application_df.drop(remove_features,1)
application_df.head()

In [ ]:
# Determine which values to replace if counts are less than 1000
replace_application = [x for x in application_df['APPLICATION_TYPE'].unique() if application_df['APPLICATION_TYPE'].value_counts()[x] < 1000]

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
replace_class = [x for x in application_df['CLASSIFICATION'].unique() if application_df['CLASSIFICATION'].value_counts()[x] < 2000]

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

In [ ]:
replace_inc = [x for x in application_df['INCOME_AMT'].unique() if application_df['INCOME_AMT'].value_counts()[x] < 20000]

# Replace in dataframe
for inc in replace_inc:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(inc,"1")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

In [ ]:
# Generate our categorical variable lists
application_cat = [cat for cat in application_df.columns if application_df[cat].dtypes == 'object']
application_cat

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat].values))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)
application_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
target = 'IS_SUCCESSFUL'
y = application_df[target]
X = application_df.drop([target],1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Train, test, evaluate model with altered data
#### Model Architecture to remain the same. 

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0,])
hidden_nodes_layer1 = 19
hidden_nodes_layer2 = 11
hidden_nodes_layer3 = 7
hidden_nodes_layer4 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1,
                            input_dim = number_input_features,
                            activation = 'relu')
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2,
                          activation='relu')
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3,
                          activation='relu')
)

# Fourth hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4,
                          activation='relu')
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1,
                          activation='sigmoid')
)

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    save_freq='epoch'
)

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

## Second Attempt
### Focus on better binning of features

In [ ]:
# Get data again into new DataFrame
application_df2 = pd.read_csv("Resources/charity_data.csv")
application_df2.head()

In [ ]:
# Perform above transformations

# Still drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df2 = application_df2.drop(['EIN','NAME'],1)
application_df2.head()

# Drop columns
remove_features = ['STATUS','SPECIAL_CONSIDERATIONS']
application_df2 = application_df2.drop(remove_features,1)
application_df2.head()

# Replace sparse values in APPLICATION_TYPE
replace_application = [x for x in application_df2['APPLICATION_TYPE'].unique() if application_df2['APPLICATION_TYPE'].value_counts()[x] < 1000]

# Replace in dataframe
for app in replace_application:
    application_df2.APPLICATION_TYPE = application_df2.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df2.APPLICATION_TYPE.value_counts()

# Replace sparse values in CLASSIFICATION
replace_class = [x for x in application_df2['CLASSIFICATION'].unique() if application_df2['CLASSIFICATION'].value_counts()[x] < 2000]

# Replace in dataframe
for cls in replace_class:
    application_df2.CLASSIFICATION = application_df2.CLASSIFICATION.replace(cls,"Other")




In [ ]:
# Bin INCOME_AMT to 0 or some amount of income.
replace_inc = [x for x in application_df2['INCOME_AMT'].unique() if application_df2['INCOME_AMT'].value_counts()[x] < 20000]

# Replace in dataframe
for inc in replace_inc:
    application_df2.INCOME_AMT = application_df2.INCOME_AMT.replace(inc,"1")
    
# Check to make sure binning was successful
application_df2.INCOME_AMT.value_counts()

In [ ]:
# Get ont hot vector of categorical variables
application_cat = [cat for cat in application_df2.columns if application_df2[cat].dtypes == 'object']
application_cat

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df2[application_cat].values))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

# Merge one-hot encoded features and drop the originals
application_df2 = application_df2.merge(encode_df,left_index=True,right_index=True).drop(application_cat,1)
application_df2.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
target = 'IS_SUCCESSFUL'
y = application_df2[target]
X = application_df2.drop([target],1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0,])
hidden_nodes_layer1 = 7
hidden_nodes_layer2 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1,
                            input_dim = number_input_features,
                            activation = 'relu')
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2,
                          activation='relu')
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1,
                          activation='sigmoid')
)

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    save_freq='epoch'
)

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

## Third attempt
### Changing model architecture only with changes to data from Attempt 2

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.iloc[0,])
hidden_nodes_layer1 = 19
hidden_nodes_layer2 = 11
hidden_nodes_layer3 = 7
hidden_nodes_layer4 = 3

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1,
                            input_dim = number_input_features,
                            activation = 'relu')
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2,
                          activation='relu')
)

# Third hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3,
                          activation='relu')
)

# Fourth hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4,
                          activation='relu')
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1,
                          activation='sigmoid')
)

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
nn.save("AlphabetSoupCharity_Optimization.h5")